In [1]:
variables = [f"X{i}" for i in range(1, 31)]
domain = {var: list(range(1, 11)) for var in variables}

In [2]:
def limit_domain(domain):
    # Обмеження: змінні на парних позиціях мають парні значення
    for var in variables:
        if int(var[1:]) % 2 == 0:
            domain[var] = list(range(2, 11, 2))

    # Обмеження: X15 = 7
    domain["X15"] = [7]
    return domain

In [3]:
domain = limit_domain(domain)
domain

{'X1': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
 'X2': [2, 4, 6, 8, 10],
 'X3': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
 'X4': [2, 4, 6, 8, 10],
 'X5': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
 'X6': [2, 4, 6, 8, 10],
 'X7': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
 'X8': [2, 4, 6, 8, 10],
 'X9': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
 'X10': [2, 4, 6, 8, 10],
 'X11': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
 'X12': [2, 4, 6, 8, 10],
 'X13': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
 'X14': [2, 4, 6, 8, 10],
 'X15': [7],
 'X16': [2, 4, 6, 8, 10],
 'X17': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
 'X18': [2, 4, 6, 8, 10],
 'X19': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
 'X20': [2, 4, 6, 8, 10],
 'X21': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
 'X22': [2, 4, 6, 8, 10],
 'X23': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
 'X24': [2, 4, 6, 8, 10],
 'X25': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
 'X26': [2, 4, 6, 8, 10],
 'X27': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
 'X28': [2, 4, 6, 8, 10],
 'X29': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
 'X30': [2, 4, 6, 8, 10]}

In [4]:
class Constraint:
    def __init__(self, variables, limit):
        self.variables = variables
        self.limit = limit

    def __str__(self):
        return f"Variables: {self.variables}, limit rule: {self.limit.__name__}\n"

In [5]:
def variables_assigned(variables, assignment):
    return all(var in assignment for var in variables)


def sum_is_multiple_of_3(variables, assignment):
    return sum(assignment[var] for var in variables) % 3 != 0


def sum_is_15(variables, assignment):
    return sum(assignment[var] for var in variables) == 15


def is_strictly_increasing(variables, assignment):
    prev_var, current_var, next_var, = variables
    return assignment[prev_var] < assignment[current_var] < assignment[next_var]


def is_distance_valid(variables, assignment):
    prev_var, current_var = variables
    return (abs(assignment[current_var] - assignment[prev_var]) > 6
            or abs(assignment[current_var] - assignment[prev_var]) < 3)


In [6]:
constraints = [
    Constraint(["X8", "X9", "X10"], sum_is_multiple_of_3),
    Constraint(["X1", "X2", "X3"], sum_is_15),
    Constraint(["X20", "X21", "X22"], is_strictly_increasing)
]

In [7]:
for i in range(1, len(variables)):
    dist_vars = variables[i - 1:i + 1]
    constraints.append(Constraint(dist_vars, is_distance_valid))

In [8]:
for constraint in constraints:
    print(constraint)

Variables: ['X8', 'X9', 'X10'], limit rule: sum_is_multiple_of_3

Variables: ['X1', 'X2', 'X3'], limit rule: sum_is_15

Variables: ['X20', 'X21', 'X22'], limit rule: is_strictly_increasing

Variables: ['X1', 'X2'], limit rule: is_distance_valid

Variables: ['X2', 'X3'], limit rule: is_distance_valid

Variables: ['X3', 'X4'], limit rule: is_distance_valid

Variables: ['X4', 'X5'], limit rule: is_distance_valid

Variables: ['X5', 'X6'], limit rule: is_distance_valid

Variables: ['X6', 'X7'], limit rule: is_distance_valid

Variables: ['X7', 'X8'], limit rule: is_distance_valid

Variables: ['X8', 'X9'], limit rule: is_distance_valid

Variables: ['X9', 'X10'], limit rule: is_distance_valid

Variables: ['X10', 'X11'], limit rule: is_distance_valid

Variables: ['X11', 'X12'], limit rule: is_distance_valid

Variables: ['X12', 'X13'], limit rule: is_distance_valid

Variables: ['X13', 'X14'], limit rule: is_distance_valid

Variables: ['X14', 'X15'], limit rule: is_distance_valid

Variables: ['X1

In [9]:
def is_consistent(var, value, assignment):
    assignment[var] = value
    result = True

    for constraint in constraints:
        if var in constraint.variables and variables_assigned(constraint.variables, assignment):
            if not constraint.limit(constraint.variables, assignment):
                result = False
                break

    del assignment[var]
    return result


# Евристики
def minimum_remaining_values(domain, assignment):
    """Еристика з мінімальною кількістю решти значень (Minimum Remaining Values - MRV). 
    Для подальшого присвоєння буде обиратися змінна у якою в області визначення буде 
    мінімальна кількість можливих значень для присвоєння."""

    unassigned = [var for var in domain if var not in assignment]

    min_var = None
    min_count = float('inf')
    for var in unassigned:
        valid_values = [
            value for value in domain[var]
            if is_consistent(var, value, assignment)
        ]
        if len(valid_values) < min_count:
            min_count = len(valid_values)
            min_var = var

    return min_var


def degree_heuristic(domain, constraints, assignment):
    """Ступенева евристика (degree) Для подальшого присвоєння буде обиратися змінна яка 
    бере участь в найбільшій кількості обмежень на інші змінні 
    з неприсвоєнними значеннями. (Це як коеф. розгалудження у графі)"""

    unassigned = [var for var in domain if var not in assignment]
    max_lim_count = {var: 0 for var in unassigned}
    for var in unassigned:
        for constraint in constraints:
            if var in constraint.variables and variables_assigned(constraint.variables, unassigned):
                max_lim_count[var] += 1

    return max(max_lim_count, key=max_lim_count.get)


def least_constraining_value(var, domain, assignment):
    """Евристика з найменш обмежувальним значенням. У ній надається перевага значенню, 
    в якому з розгляду виключається найменша
    кількість варіантів вибору значень для сусідніх змінних в графі обмежень."""

    def count_conflicts(value):
        assignment[var] = value
        conflicts = 0

        idx = variables.index(var)
        neighbors = []

        if idx > 0:
            neighbors.append(variables[idx - 1])

        if idx < len(domain) - 1:
            neighbors.append(variables[idx + 1])

        for neighbor in neighbors:
            if neighbor not in assignment:
                if not any(is_consistent(neighbor, v, assignment) for v in domain[neighbor]):
                    conflicts += 1
        del assignment[var]
        return conflicts

    return sorted(domain[var], key=count_conflicts)


def backtracking_search(domain):
    """Пошук з поверненням - це Пошук в глибину, в якому кожен раз вибираються значення для 
    однієї змінної і виконується повернення, якщо більше не залишається допустимих значень,
    які можна було б присвоїти змінній."""

    assignment = {}

    def backtrack():
        if len(assignment) == len(variables):
            return assignment

        var = minimum_remaining_values(domain, assignment)
        if not var:
            return None

        values = least_constraining_value(var, domain, assignment)
        for value in values:
            if is_consistent(var, value, assignment):
                assignment[var] = value
                result = backtrack()
                if result:
                    return result
                del assignment[var]

        return None

    return backtrack()


solution = backtracking_search(domain)
if solution:
    print("Знайдено розв'язок:")
    print(solution, "\n")
    print({k: v for k, v in sorted(solution.items(), key=lambda item: int(item[0][1:]))})
else:
    print("Розв'язок не знайдено.")


Знайдено розв'язок:
{'X15': 7, 'X14': 6, 'X16': 6, 'X2': 2, 'X4': 2, 'X6': 2, 'X8': 2, 'X10': 2, 'X9': 1, 'X12': 2, 'X13': 4, 'X17': 4, 'X18': 2, 'X20': 2, 'X22': 4, 'X21': 3, 'X23': 2, 'X24': 2, 'X26': 2, 'X28': 2, 'X30': 2, 'X1': 3, 'X3': 10, 'X5': 1, 'X7': 1, 'X11': 1, 'X19': 1, 'X25': 1, 'X27': 1, 'X29': 1} 

{'X1': 3, 'X2': 2, 'X3': 10, 'X4': 2, 'X5': 1, 'X6': 2, 'X7': 1, 'X8': 2, 'X9': 1, 'X10': 2, 'X11': 1, 'X12': 2, 'X13': 4, 'X14': 6, 'X15': 7, 'X16': 6, 'X17': 4, 'X18': 2, 'X19': 1, 'X20': 2, 'X21': 3, 'X22': 4, 'X23': 2, 'X24': 2, 'X25': 1, 'X26': 2, 'X27': 1, 'X28': 2, 'X29': 1, 'X30': 2}
